In [1]:
import pandas as pd

df = pd.read_excel('answers.xlsx')
print(df)

            ID       Время создания      Время изменения  \
0   1311191306  2023-02-12 16:46:53  2023-02-12 16:46:53   
1   1311025212  2023-02-12 11:27:25  2023-02-12 11:27:25   
2   1310845113  2023-02-11 23:16:43  2023-02-11 23:16:43   
3   1309216774  2023-02-09 16:45:29  2023-02-09 16:45:29   
4   1308822969  2023-02-09 08:13:56  2023-02-09 08:13:56   
5   1308696138  2023-02-08 22:57:10  2023-02-08 22:57:10   
6   1308515251  2023-02-08 18:02:50  2023-02-08 18:02:50   
7   1308256077  2023-02-08 12:43:25  2023-02-08 12:43:25   
8   1308232752  2023-02-08 12:14:02  2023-02-08 12:14:02   
9   1308216282  2023-02-08 11:53:11  2023-02-08 11:53:11   
10  1308196582  2023-02-08 11:28:42  2023-02-08 11:28:42   
11  1308168244  2023-02-08 10:51:07  2023-02-08 10:51:07   
12  1308161303  2023-02-08 10:41:45  2023-02-08 10:41:45   
13  1308159865  2023-02-08 10:39:48  2023-02-08 10:39:48   
14  1308158623  2023-02-08 10:38:07  2023-02-08 10:38:07   
15  1308157842  2023-02-08 10:37:04  202

In [2]:
df.columns

Index(['ID', 'Время создания', 'Время изменения',
       'Расскажите, что вам важно в организации, когда вы решаете, жертвовать ей, или нет? / Размер организации',
       'Расскажите, что вам важно в организации, когда вы решаете, жертвовать ей, или нет? / Качественные отчеты',
       'Расскажите, что вам важно в организации, когда вы решаете, жертвовать ей, или нет? / Хорошие отзывы',
       'Расскажите, что вам важно в организации, когда вы решаете, жертвовать ей, или нет? / Множество филиалов',
       'Расскажите, что вам важно в организации, когда вы решаете, жертвовать ей, или нет? / Географическое расположение организации',
       'Расскажите, что вам важно в организации, когда вы решаете, жертвовать ей, или нет? / Разнообразие кампаний по сбору средств',
       'Расскажите, что вам важно в организации, когда вы решаете, жертвовать ей, или нет? / Статус иноагента/его отсутствие',
       'Расскажите, на основе чего еще вы делаете свой выбор?',
       'Расскажите, на чем специализи

In [3]:
# first step: transform table into something readable

#output format: list of dicts

#
# importance order: [Размер, Отчеты, Отзывы, Филиалы, Расположение, Кампании]
# tags order: [Работа с детьми, Борьба с бедностью, Наука и исследования, 
# Искусство, Здравоохранение, Образование, Работа с животными, Обеспечение прав и свобод человека]
#
# { 'organisations' : [['Благотворительная организация «Ночлежка»', 100], ['ОВД-Инфо', 200], [...] ...], 
#'importance': [1, 1, 0, 0, 1, 0]}
#'additional_importance' : 'Наличие на сайте информации о проведенных кампаниях: на что потрачены деньги, что получилось'
#'tags': [1, 0, 0, 1, 0, 1, 1, 1]
#'additional_tags': 'Экология' }

In [23]:
import math
import time
import re
import random
import string
print(df.shape)
print(type(df['Расскажите, куда еще вы уже жертвовали'][2]))

(17, 63)
<class 'float'>


In [10]:
from google.cloud import bigquery
client = bigquery.Client(project='donapp-d2378')
dataset_id = 'firebase_recommendations_dataset'
# For this sample, the table must already exist and have a defined schematable_id = ‘test_table_creation’
table_ref = client.dataset(dataset_id).table('donations_table')
table = client.get_table(table_ref)
# Creating a list of tuples with the values that shall be inserted into the table

In [11]:
import firebase_admin
from firebase_admin import firestore
from firebase_admin import credentials

# Use the application default credentials.
cred = credentials.Certificate('donapp-d2378-firebase-adminsdk-zxd1d-2147e3a97f.json') 
app = firebase_admin.initialize_app(cred)

db = firestore.client()

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [58]:
rows_to_insert = []
def insert_into_table(userID, primaryid, amount, timestamp):
    rows_to_insert.append({
    u'userID': userID,
    u'charityID': primaryid,
    u'amount': amount,
    u'timestamp': timestamp,
    })

confusingcharities = []
confusingamounts = []


def add_entry(charityname, amount, userID, timestamp):
    print(userID, '|', charityname, '|', amount, '|', timestamp)
    if isinstance(charityname, list):
        for i in range(len(charityname)):
            if isinstance(amount, list):
                if (i < len(amount)):   
                    add_entry(charityname[i], amount, userID, timestamp)
                else:
                    add_entry(charityname[i], 'nan', userID, timestamp)
            else:
                add_entry(charityname[i], 'nan', userID, timestamp)
        return
    
    charities = db.collection(u'charityaliases')
    lookupref = charities.document(charityname)
    doc = lookupref.get()
    primaryname = ''
    primaryid = ''
    if not doc.exists:
        print('what is: ' + charityname)
        primaryname = input()
        if primaryname == '?':
            confusingcharities.append(charityname)
            confusingamounts.append(confusingamounts)
            return
        primarydoc = charities.document(primaryname).get()
        if not primarydoc.exists:
            random.seed(timestamp)
            cid = ''.join(random.choices(string.ascii_uppercase + string.digits + string.ascii_lowercase, k=28))
            charities.document(primaryname).set({'primary': primaryname, 'id': cid})
            primaryid = cid
        else:
            primaryid = primarydoc.to_dict()['id']
        charities.document(charityname).set({'primary': primaryname, 'id': primaryid})
    else:
        primaryname = doc.to_dict()['primary']
        primaryid = doc.to_dict()['id']
    
    iamount = 0
    if not amount == 'nan':
        try:
            iamount = int(amount)
            amount = str(iamount)
        except:
            print('how much is: ', amount)
            amount = input()
            if amount == '?':
                amount = 'nan'
    
    insert_into_table(userID, primaryid, amount, timestamp)

    

In [59]:
votes = []
already = ['Расскажите, куда вы уже жертвовали',
       'Расскажите, сколько (необязательно)',
       'Расскажите, куда еще вы уже жертвовали',
       'Расскажите, сколько (необязательно).1',
       'Расскажите, куда еще вы уже жертвовали.1',
       'Расскажите, сколько (необязательно).2',
       'Расскажите, куда еще вы уже жертвовали.2',
       'Расскажите, сколько (необязательно).3',
       'Расскажите, куда еще вы уже жертвовали.3',
       'Расскажите, сколько (необязательно).4',
       'Расскажите, куда еще вы уже жертвовали.4',
       'Расскажите, сколько (необязательно).5',
       'Расскажите, куда еще вы уже жертвовали.5',
       'Расскажите, сколько (необязательно).6',
       'Расскажите, куда еще вы уже жертвовали.6',
       'Расскажите, сколько (необязательно).7',
       'Расскажите, куда еще вы уже жертвовали.7',
       'Расскажите, сколько (необязательно).8',
       'Расскажите, куда еще вы уже жертвовали.8',
       'Расскажите, сколько (необязательно).9',
       'ОГО! Вы по-настоящему щедрый человек! Перечислите, пожалуйста, куда еще вы жертвовали, здесь']

planned = ['Расскажите, куда вы хотели бы пожертвовать',
       'Расскажите, сколько (необязательно).10',
       'Расскажите, куда еще вы хотели бы пожертвовать',
       'Расскажите, сколько (необязательно).11',
       'Расскажите, куда еще вы хотели бы пожертвовать.1',
       'Расскажите, сколько (необязательно).12',
       'Расскажите, куда еще вы хотели бы пожертвовать.2',
       'Расскажите, сколько (необязательно).13',
       'Расскажите, куда еще вы хотели бы пожертвовать.3',
       'Расскажите, сколько (необязательно).14',
       'Расскажите, куда еще вы хотели бы пожертвовать.4',
       'Расскажите, сколько (необязательно).15',
       'Расскажите, куда еще вы хотели бы пожертвовать.5',
       'Расскажите, сколько (необязательно).16',
       'Расскажите, куда еще вы хотели бы пожертвовать.6',
       'Расскажите, сколько (необязательно).17',
       'Расскажите, куда еще вы хотели бы пожертвовать.7',
       'Расскажите, сколько (необязательно).18',
       'Расскажите, куда еще вы хотели бы пожертвовать.8',
       'Расскажите, сколько (необязательно).19',
       'ОГО! У вас много планов! Перечислите, пожалуйста, куда еще вы хотели бы пожертвовать, здесь']

def send_charities(charitynames, amounts, userID, timestamp):
    for j in range(len(charitynames)):
        if j >= len(amounts):
            add_entry(charitynames[j], 'nan', userID, timestamp)
        else:   
            add_entry(charitynames[j], amounts[j], userID, timestamp)
        timestamp += 1
    return timestamp

def parse_columns(nametables):
    random.seed(0)
    timestamp = int(time.time())
    for i in range(df.shape[0]):
        newUserId = ''.join(random.choices(string.ascii_uppercase + string.digits + string.ascii_lowercase, k=28))
        newUserId
        j = 0

        if isinstance(df[nametables[j]][i], float):
            continue

        charitynames =  df[nametables[j]][i].split(',')
        if isinstance(df[nametables[j + 1]][i], float):
            amounts = []
        else:
            amounts = df[nametables[j + 1]][i].split(',')
        if not len(charitynames) == 1:
            timestamp = send_charities(charitynames, amounts, newUserId, timestamp)

        while j < 20:
            charityname = df[nametables[j]][i]
            if isinstance(charityname, float):
                break
            amount = df[nametables[j + 1]][i]
            add_entry(charitynames, amount, newUserId, timestamp)
            j += 2
            timestamp += 1

        if j == 20:
            charitynames = df[nametables[j]][i].split(',')
            timestamp = send_charities(charitynames, [], newUserID, timestamp)
            
parse_columns(already)
parse_columns(planned)


errors = client.insert_rows(table, rows_to_insert) 
print(errors)

qk0Q5ZmS3au5RkcPuyotTjtg3G0b | Дари Еду | Дари Еду - подписка | 1678288413
how much is:  Дари Еду - подписка
?
qk0Q5ZmS3au5RkcPuyotTjtg3G0b |  РусДог |  200 р в месяц. РусДог и небольшие приюты - разово в зависимости от кампании | 1678288414
how much is:   200 р в месяц. РусДог и небольшие приюты - разово в зависимости от кампании
400
qk0Q5ZmS3au5RkcPuyotTjtg3G0b |  небольшие приюты для животных |  300-1000 рублей. WWF - покупка мерча | 1678288415
what is:  небольшие приюты для животных
?
qk0Q5ZmS3au5RkcPuyotTjtg3G0b |  WWF |  2000 р | 1678288416
how much is:   2000 р
4000
qk0Q5ZmS3au5RkcPuyotTjtg3G0b | ['Дари Еду', ' РусДог', ' небольшие приюты для животных', ' WWF'] | Дари Еду - подписка, 200 р в месяц. РусДог и небольшие приюты - разово в зависимости от кампании, 300-1000 рублей. WWF - покупка мерча, 2000 р | 1678288417
qk0Q5ZmS3au5RkcPuyotTjtg3G0b | Дари Еду | nan | 1678288417
qk0Q5ZmS3au5RkcPuyotTjtg3G0b |  РусДог | nan | 1678288417
qk0Q5ZmS3au5RkcPuyotTjtg3G0b |  небольшие приюты

?
KQFfe5lN41XR3ltWHZWYbweXOiQs | ['Фонды', ' которые противодействуют современному рабству! Раньше был фонд Альтернатива', ' сейчас совсем непонятно', ' пытается ли кто-то с этой проблемой бороться'] | nan | 1678288844
KQFfe5lN41XR3ltWHZWYbweXOiQs | Фонды | nan | 1678288844
what is: Фонды
?
KQFfe5lN41XR3ltWHZWYbweXOiQs |  которые противодействуют современному рабству! Раньше был фонд Альтернатива | nan | 1678288844
KQFfe5lN41XR3ltWHZWYbweXOiQs |  сейчас совсем непонятно | nan | 1678288844
what is:  сейчас совсем непонятно
?
KQFfe5lN41XR3ltWHZWYbweXOiQs |  пытается ли кто-то с этой проблемой бороться | nan | 1678288844
what is:  пытается ли кто-то с этой проблемой бороться
?
IgTNwJa4DlZJdlTRauS8OLiPTdhy | ['Гражданское содействие'] | nan | 1678288845
IgTNwJa4DlZJdlTRauS8OLiPTdhy | Гражданское содействие | nan | 1678288845
Vo2lVcrAMbW4NFm8BPlFxV9tCFJ0 | никуда | nan | 1678288846
what is: никуда
?
Vo2lVcrAMbW4NFm8BPlFxV9tCFJ0 |  я плохой злой человек | nan | 1678288847
what is:  я плохой 